# GWEMFISH Input Section - Jupyter Notebook Example

This notebook demonstrates how to use the GWEMFISH input functionality to define lens models and parameters for gravitational lensing analysis.

## Table of Contents
1. [Setup and Imports](#setup)
2. [Basic Lens Model Setup](#basic)
3. [Advanced Multi-Component Lens](#advanced)
4. [Parameter Management](#parameters)
5. [Input Validation](#validation)
6. [Exporting for Analysis](#export)


## 1. Setup and Imports {#setup}

First, let's import the necessary modules and set up the path to the GWEMFISH package.


In [25]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add the GWEMFISH package to the path
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

# Import GWEMFISH input components
from gwemfish.input.lens_input import LensInput
from gwemfish.input.parameter_input import ParameterInput
from gwemfish.models.lens_models import SupportedLensModels

print("✓ Imports successful!")
print(f"✓ GWEMFISH package path: {os.path.join(os.getcwd(), '..')}")


✓ Imports successful!
✓ GWEMFISH package path: /Users/souvik/Documents/herculens_project/GWEMFISH/examples/..


## 2. Basic Lens Model Setup {#basic}

Let's start with a simple SIE (Singular Isothermal Ellipsoid) lens model.

**Important Note**: In gravitational wave analysis, the source position is treated as separate parameters `y1_gw` and `y2_gw` rather than a single position tuple. This allows for proper parameter estimation and uncertainty quantification in the GW analysis pipeline.


In [ ]:
# Create a basic lens input
lens_input = LensInput()

# Add a simple SIE lens model
# Note: Parameters in 'parameters' dict are free, 'fixed_parameters' are fixed
lens_input.add_lens_model(
    model_type="SIE",
    parameters={
        'theta_E': 5.0,      # Einstein radius in arcsec (FREE parameter)
        'e1': 0.137,         # Ellipticity component 1 (FREE parameter)
        'e2': 0.039          # Ellipticity component 2 (FREE parameter)
    },
    fixed_parameters={
        'center_x': 0.0,     # Fixed at origin (FIXED parameter)
        'center_y': 0.0      # Fixed at origin (FIXED parameter)
    }
)

print("✓ Basic SIE lens model added")


✓ Basic SIE lens model added


In [27]:
# Set up the source
lens_input.set_source(
    model_type="SersicElliptic",
    parameters={
        'amp': 4.0,          # Source amplitude
        'R_sersic': 0.5,     # Sersic radius
        'n_sersic': 2.0,     # Sersic index
        'e1': 0.05,          # Source ellipticity
        'e2': 0.05
    },
    fixed_parameters={
        'center_x': 0.05,    # Source position
        'center_y': 0.1
    },
    redshift=2.0             # Source redshift
)

# Set up GW source position parameters separately
gw_source_params = {
    'y1_gw': 0.3,            # Source plane y1 position (GW parameter)
    'y2_gw': 0.2             # Source plane y2 position (GW parameter)
}

print("✓ Source model added")
print("✓ GW source position parameters defined separately")


✓ Source model added
✓ GW source position parameters defined separately


In [ ]:
# Alternative: Fixed cosmology parameters
# If you want to fix cosmology parameters instead of treating them as free:
# lens_input.set_cosmology(
#     H0=67.3,
#     Om0=0.316,
#     use_as_parameters=False  # Fix at these values
# )

print("Note: Other parameters (lens, source, etc.) are already defined as free parameters")


In [ ]:
# Set up cosmology
# Note: use_as_parameters=True means H0 and Om0 will be treated as free parameters
# If use_as_parameters=False, they would be fixed at the given values
lens_input.set_cosmology(
    H0=67.3,                # Hubble constant (km/s/Mpc)
    Om0=0.316,              # Matter density parameter
    use_as_parameters=True  # Treat as free parameters for estimation
)

print("✓ Cosmology parameters set as free parameters")


✓ Cosmology parameters set


In [29]:
# Set up observation parameters
lens_input.set_observation(
    pixel_scale=0.08,        # arcsec/pixel
    image_size=(200, 200),   # pixels
    psf_fwhm=0.3,           # arcsec
    background_rms=0.01      # Background noise
)

print("✓ Observation parameters set")


✓ Observation parameters set


In [30]:
# Set image positions (these would normally be computed from source position)
image_positions = [
    (0.94, 5.33),           # Image 1
    (2.90, -4.22),          # Image 2
    (5.02, -0.44),          # Image 3
    (-4.26, -1.10)          # Image 4
]

lens_input.set_image_positions(image_positions)

print(f"✓ Image positions set: {len(image_positions)} images")
for i, (x, y) in enumerate(image_positions):
    print(f"  Image {i+1}: ({x:.2f}, {y:.2f}) arcsec")


✓ Image positions set: 4 images
  Image 1: (0.94, 5.33) arcsec
  Image 2: (2.90, -4.22) arcsec
  Image 3: (5.02, -0.44) arcsec
  Image 4: (-4.26, -1.10) arcsec


## 3. Advanced Multi-Component Lens {#advanced}

Now let's create a more complex lens model with multiple components.


In [7]:
# Create a new lens input for advanced example
advanced_lens = LensInput()

# Add main SIE lens
advanced_lens.add_lens_model(
    model_type="SIE",
    parameters={
        'theta_E': 5.0,
        'e1': 0.137,
        'e2': 0.039
    },
    fixed_parameters={
        'center_x': 0.0,
        'center_y': 0.0
    }
)

# Add external shear
advanced_lens.add_lens_model(
    model_type="SHEAR",
    parameters={
        'gamma1': -3.09e-5,  # Small external shear
        'gamma2': 9.51e-5
    },
    fixed_parameters={
        'ra_0': 0.0,
        'dec_0': 0.0
    }
)

# Add external convergence
advanced_lens.add_lens_model(
    model_type="CONVERGENCE",
    parameters={
        'kappa_ext': 0.01    # Small external convergence
    },
    fixed_parameters={
        'ra_0': 0.0,
        'dec_0': 0.0
    }
)

print("✓ Advanced multi-component lens model created")
print(f"  Lens models: {advanced_lens.get_lens_model_list()}")


✓ Advanced multi-component lens model created
  Lens models: ['SIE', 'SHEAR', 'CONVERGENCE']


In [31]:
# Set up source with more complex parameters
advanced_lens.set_source(
    model_type="SersicElliptic",
    parameters={
        'amp': 4.0,
        'R_sersic': 0.5,
        'n_sersic': 2.0,
        'e1': 0.05,
        'e2': 0.05
    },
    fixed_parameters={
        'center_x': 0.05,
        'center_y': 0.1
    },
    redshift=2.0
)

# Set up GW source position parameters separately
advanced_gw_source_params = {
    'y1_gw': 0.3,            # Source plane y1 position (GW parameter)
    'y2_gw': 0.2             # Source plane y2 position (GW parameter)
}

# Add lens light
advanced_lens.set_lens_light(
    model_type="SersicElliptic",
    parameters={
        'amp': 8.0,
        'R_sersic': 1.0,
        'n_sersic': 3.0,
        'e1': 0.137,         # Aligned with lens mass
        'e2': 0.039
    },
    fixed_parameters={
        'center_x': 0.0,
        'center_y': 0.0
    }
)

print("✓ Source and lens light models added")


✓ Source and lens light models added


In [32]:
# Complete the advanced setup
advanced_lens.set_cosmology(H0=67.3, Om0=0.316, use_as_parameters=True)
advanced_lens.set_observation(
    pixel_scale=0.08,
    image_size=(200, 200),
    psf_fwhm=0.3,
    background_rms=0.01
)
advanced_lens.set_image_positions(image_positions)

print("✓ Advanced lens setup complete")


✓ Advanced lens setup complete


## 4. Parameter Management {#parameters}

Now let's set up detailed parameter management with bounds and priors.


In [33]:
# Create parameter input for detailed parameter management
param_input = ParameterInput()

# Add cosmology parameters with bounds and priors
param_input.add_cosmology_parameters(
    H0=67.3,
    Om0=0.316,
    use_as_parameters=True
)

print("✓ Cosmology parameters added")


✓ Cosmology parameters added


### Image Position Parameters: Free vs Fixed

**Current Setup**: Image positions are treated as **FREE parameters** by default.

**To Fix Image Positions**: If you want to fix the image positions instead of treating them as free parameters, you can do this:


In [47]:
# Current approach: FREE image position parameters
print("Current approach - FREE image position parameters:")
print("param_input.add_image_position_parameters(")
print("    positions=image_positions,")
print("    bounds=(-10.0, 10.0)  # Bounds for position parameters")
print(")")

print("\n" + "="*50)

# Alternative approach: FIXED image position parameters
print("Alternative approach - FIXED image position parameters:")
print("for i, (x, y) in enumerate(image_positions):")
print("    param_input.add_parameter(")
print("        name=f'image_x{i+1}',")
print("        value=x,")
print("        is_fixed=True,  # ← This makes it FIXED")
print("        description=f'Image {i+1} x position (arcsec) - FIXED',")
print("        group='image_positions'")
print("    )")
print("    # Same for image_y{i+1}")

print("\n" + "="*50)
print("Key difference:")
print("- FREE parameters: Can be optimized/estimated")
print("- FIXED parameters: Stay at the given values")


Current approach - FREE image position parameters:
param_input.add_image_position_parameters(
    positions=image_positions,
    bounds=(-10.0, 10.0)  # Bounds for position parameters
)

Alternative approach - FIXED image position parameters:
for i, (x, y) in enumerate(image_positions):
    param_input.add_parameter(
        name=f'image_x{i+1}',
        value=x,
        is_fixed=True,  # ← This makes it FIXED
        description=f'Image {i+1} x position (arcsec) - FIXED',
        group='image_positions'
    )
    # Same for image_y{i+1}

Key difference:
- FREE parameters: Can be optimized/estimated
- FIXED parameters: Stay at the given values


In [48]:
# Example: Create a parameter input with FIXED image positions
fixed_param_input = ParameterInput()

# Add other parameters first
fixed_param_input.add_cosmology_parameters(H0=67.3, Om0=0.316, use_as_parameters=True)

# Add FIXED image position parameters
print("Creating FIXED image position parameters:")
for i, (x, y) in enumerate(image_positions):
    fixed_param_input.add_parameter(
        name=f"image_x{i+1}",
        value=x,
        is_fixed=True,  # This makes it FIXED
        description=f"Image {i+1} x position (arcsec) - FIXED",
        group="image_positions"
    )
    fixed_param_input.add_parameter(
        name=f"image_y{i+1}",
        value=y,
        is_fixed=True,  # This makes it FIXED
        description=f"Image {i+1} y position (arcsec) - FIXED",
        group="image_positions"
    )
    print(f"  Fixed image_{i+1}: ({x:.2f}, {y:.2f})")

# Check the results
free_params = fixed_param_input.get_free_parameter_values()
fixed_params = fixed_param_input.get_fixed_parameter_values()

print(f"\nResults:")
print(f"  Free parameters: {len(free_params)}")
print(f"  Fixed parameters: {len(fixed_params)}")
print(f"  Fixed image positions: {[k for k in fixed_params.keys() if 'image_' in k]}")


Creating FIXED image position parameters:
  Fixed image_1: (0.94, 5.33)
  Fixed image_2: (2.90, -4.22)
  Fixed image_3: (5.02, -0.44)
  Fixed image_4: (-4.26, -1.10)

Results:
  Free parameters: 2
  Fixed parameters: 8
  Fixed image positions: ['image_x1', 'image_y1', 'image_x2', 'image_y2', 'image_x3', 'image_y3', 'image_x4', 'image_y4']


## General Parameter Definition - All Possible Options

The `add_parameter()` method is available for **ALL parameters** in GWEMFISH. Here's the complete signature and all possible kwargs:


In [49]:
# General Parameter Definition Method
# param_input.add_parameter(
#     name: str,                    # REQUIRED: Parameter name
#     value: float,                 # REQUIRED: Parameter value
#     bounds: Optional[Tuple[float, float]] = None,  # Optional: (lower, upper) bounds
#     prior: Optional[Dict[str, Any]] = None,        # Optional: Prior distribution
#     is_fixed: bool = False,       # Optional: FREE (False) or FIXED (True)
#     description: str = "",        # Optional: Human-readable description
#     group: str = "lens"           # Optional: Parameter group
# )

print("Complete parameter definition method signature:")
print("=" * 60)
print("param_input.add_parameter(")
print("    name='parameter_name',           # REQUIRED")
print("    value=1.0,                      # REQUIRED")
print("    bounds=(0.0, 10.0),             # Optional: (lower, upper)")
print("    prior={'distribution': 'normal', 'parameters': {'loc': 1.0, 'scale': 0.1}},  # Optional")
print("    is_fixed=False,                 # Optional: FREE (False) or FIXED (True)")
print("    description='Parameter description',  # Optional")
print("    group='lens'                    # Optional: cosmology, lens, source, etc.")
print(")")


Complete parameter definition method signature:
param_input.add_parameter(
    name='parameter_name',           # REQUIRED
    value=1.0,                      # REQUIRED
    bounds=(0.0, 10.0),             # Optional: (lower, upper)
    prior={'distribution': 'normal', 'parameters': {'loc': 1.0, 'scale': 0.1}},  # Optional
    is_fixed=False,                 # Optional: FREE (False) or FIXED (True)
    description='Parameter description',  # Optional
    group='lens'                    # Optional: cosmology, lens, source, etc.
)


In [50]:
# Examples of different parameter types using add_parameter()

# Create a new parameter input for examples
example_param_input = ParameterInput()

print("EXAMPLES OF DIFFERENT PARAMETER TYPES:")
print("=" * 50)

# 1. FREE parameter with bounds and prior
example_param_input.add_parameter(
    name="custom_lens_param",
    value=2.5,
    bounds=(0.1, 5.0),
    prior={'distribution': 'normal', 'parameters': {'loc': 2.0, 'scale': 0.5}},
    is_fixed=False,
    description="Custom lens parameter for testing",
    group="lens"
)

# 2. FIXED parameter (no bounds/prior needed)
example_param_input.add_parameter(
    name="fixed_cosmology_param",
    value=0.7,
    is_fixed=True,
    description="Fixed cosmology parameter",
    group="cosmology"
)

# 3. FREE parameter with uniform prior
example_param_input.add_parameter(
    name="uniform_source_param",
    value=1.0,
    bounds=(0.0, 2.0),
    prior={'distribution': 'uniform', 'parameters': {'low': 0.0, 'high': 2.0}},
    description="Source parameter with uniform prior",
    group="source"
)

# 4. FREE parameter with lognormal prior
example_param_input.add_parameter(
    name="lognormal_amplitude",
    value=10.0,
    bounds=(0.1, 100.0),
    prior={'distribution': 'lognormal', 'parameters': {'loc': 2.0, 'scale': 0.5}},
    description="Amplitude parameter with lognormal prior",
    group="source"
)

# 5. FREE parameter with truncated normal prior
example_param_input.add_parameter(
    name="truncated_ellipticity",
    value=0.1,
    bounds=(-0.5, 0.5),
    prior={'distribution': 'truncated_normal', 'parameters': {'loc': 0.0, 'scale': 0.1, 'low': -0.5, 'high': 0.5}},
    description="Ellipticity parameter with truncated normal prior",
    group="lens"
)

# 6. Minimal parameter (only name and value)
example_param_input.add_parameter(
    name="minimal_param",
    value=3.14
)

print("✓ Added 6 different parameter types")
print(f"Total parameters: {len(example_param_input.get_parameter_values())}")
print(f"Free parameters: {len(example_param_input.get_free_parameter_values())}")
print(f"Fixed parameters: {len(example_param_input.get_fixed_parameter_values())}")


EXAMPLES OF DIFFERENT PARAMETER TYPES:
✓ Added 6 different parameter types
Total parameters: 6
Free parameters: 5
Fixed parameters: 1


In [ ]:
# Show all possible prior distributions
print("\nSUPPORTED PRIOR DISTRIBUTIONS:")
print("=" * 40)

prior_examples = {
    "uniform": {
        'distribution': 'uniform', 
        'parameters': {'low': 0.0, 'high': 1.0}
    },
    "normal": {
        'distribution': 'normal', 
        'parameters': {'loc': 0.0, 'scale': 1.0}
    },
    "lognormal": {
        'distribution': 'lognormal', 
        'parameters': {'loc': 0.0, 'scale': 1.0}
    },
    "truncated_normal": {
        'distribution': 'truncated_normal', 
        'parameters': {'loc': 0.0, 'scale': 1.0, 'low': -2.0, 'high': 2.0}
    }
}

for dist_name, prior_dict in prior_examples.items():
    print(f"{dist_name.upper()}:")
    print(f"  {prior_dict}")
    print()

print("SUPPORTED PARAMETER GROUPS:")
print("=" * 30)
groups = ['cosmology', 'lens', 'source', 'lens_light', 'image_positions', 'noise', 'redshifts', 'gw_source']
for group in groups:
    print(f"  - {group}")

print("\nPARAMETER STATUS OPTIONS:")
print("=" * 30)
print("  - is_fixed=False  → FREE parameter (can be optimized)")
print("  - is_fixed=True   → FIXED parameter (stays at given value)")


In [51]:
# Show the detailed parameter information
print("\nDETAILED PARAMETER INFORMATION:")
print("=" * 40)
example_param_input.print_summary()



DETAILED PARAMETER INFORMATION:
GWEMFISH Parameter Input Summary

Cosmology Parameters:
  fixed_cosmology_param: 0.700000 (FIXED)
    Fixed cosmology parameter

Lens Parameters:
  custom_lens_param: 2.500000 (FREE) [0.100, 5.000]
    Custom lens parameter for testing
  truncated_ellipticity: 0.100000 (FREE) [-0.500, 0.500]
    Ellipticity parameter with truncated normal prior
  minimal_param: 3.140000 (FREE)

Source Parameters:
  uniform_source_param: 1.000000 (FREE) [0.000, 2.000]
    Source parameter with uniform prior
  lognormal_amplitude: 10.000000 (FREE) [0.100, 100.000]
    Amplitude parameter with lognormal prior

Total Parameters: 6
Free Parameters: 5
Fixed Parameters: 1


In [34]:
# Add lens parameters for each model
param_input.add_lens_parameters(
    model_type="SIE",
    parameters={
        'theta_E': 5.0,
        'e1': 0.137,
        'e2': 0.039
    },
    fixed_parameters={
        'center_x': 0.0,
        'center_y': 0.0
    },
    model_index=0
)

param_input.add_lens_parameters(
    model_type="SHEAR",
    parameters={
        'gamma1': -3.09e-5,
        'gamma2': 9.51e-5
    },
    fixed_parameters={
        'ra_0': 0.0,
        'dec_0': 0.0
    },
    model_index=1
)

param_input.add_lens_parameters(
    model_type="CONVERGENCE",
    parameters={
        'kappa_ext': 0.01
    },
    fixed_parameters={
        'ra_0': 0.0,
        'dec_0': 0.0
    },
    model_index=2
)

print("✓ Lens parameters added")


✓ Lens parameters added


In [35]:
# Add image position parameters
param_input.add_image_position_parameters(
    positions=image_positions,
    bounds=(-10.0, 10.0)  # Bounds for position parameters
)

# Add source parameters
param_input.add_source_parameters(
    model_type="SersicElliptic",
    parameters={
        'amp': 4.0,
        'R_sersic': 0.5,
        'n_sersic': 2.0,
        'e1': 0.05,
        'e2': 0.05
    },
    fixed_parameters={
        'center_x': 0.05,
        'center_y': 0.1
    }
)

# Add GW source position parameters separately
param_input.add_parameter(
    name="y1_gw",
    value=0.3,
    bounds=(-1.0, 1.0),
    prior={'distribution': 'uniform', 'parameters': {'low': -1.0, 'high': 1.0}},
    description="Source plane y1 position (GW parameter)",
    group="gw_source"
)

param_input.add_parameter(
    name="y2_gw",
    value=0.2,
    bounds=(-1.0, 1.0),
    prior={'distribution': 'uniform', 'parameters': {'low': -1.0, 'high': 1.0}},
    description="Source plane y2 position (GW parameter)",
    group="gw_source"
)

# Add lens light parameters
param_input.add_lens_light_parameters(
    model_type="SersicElliptic",
    parameters={
        'amp': 8.0,
        'R_sersic': 1.0,
        'n_sersic': 3.0,
        'e1': 0.137,
        'e2': 0.039
    },
    fixed_parameters={
        'center_x': 0.0,
        'center_y': 0.0
    }
)

print("✓ Source and lens light parameters added")


✓ Source and lens light parameters added


In [36]:
# Add redshift and noise parameters
param_input.add_redshift_parameters(
    z_lens=0.5,
    z_source=2.0
)

param_input.add_noise_parameter(
    background_rms=0.01,
    bounds=(1e-3, 1e-1)  # Bounds for noise parameter
)

print("✓ Redshift and noise parameters added")


✓ Redshift and noise parameters added


## 5. Input Validation {#validation}

Let's validate our input configurations and explore the supported models.


In [37]:
# Validate the basic lens input
try:
    lens_input.validate_input()
    print("✓ Basic lens input validation passed")
except Exception as e:
    print(f"✗ Basic lens input validation failed: {e}")


✓ Basic lens input validation passed


In [38]:
# Validate the advanced lens input
try:
    advanced_lens.validate_input()
    print("✓ Advanced lens input validation passed")
except Exception as e:
    print(f"✗ Advanced lens input validation failed: {e}")


✓ Advanced lens input validation passed


In [39]:
# Explore supported lens models
print("Supported lens models:")
models = SupportedLensModels.get_supported_models()
print(f"Total: {len(models)} models")
print("\nFirst 10 models:")
for i, model in enumerate(models[:10]):
    print(f"  {i+1:2d}. {model}")
print("  ...")


Supported lens models:
Total: 27 models

First 10 models:
   1. SIE
   2. SIS
   3. NFW
   4. SHEAR
   5. CONVERGENCE
   6. POINT_MASS
   7. SPEMD
   8. SPEP
   9. CHAMELEON
  10. DPL
  ...


In [40]:
# Get detailed information about SIE model
print("SIE Model Information:")
SupportedLensModels.print_model_info('SIE')


SIE Model Information:
Lens Model: SIE
Description: Singular Isothermal Ellipsoid
Herculens name: SIE
Required parameters: ['theta_E']
Optional parameters: ['e1', 'e2', 'center_x', 'center_y']
All parameters: ['theta_E', 'e1', 'e2', 'center_x', 'center_y']


In [41]:
# Test parameter validation
print("Parameter Validation Test:")

# Valid parameters
valid_params = {
    'theta_E': 5.0,
    'e1': 0.137,
    'e2': 0.039,
    'center_x': 0.0,
    'center_y': 0.0
}

try:
    SupportedLensModels.validate_parameters('SIE', valid_params)
    print("✓ Valid SIE parameters accepted")
except ValueError as e:
    print(f"✗ Valid SIE parameters rejected: {e}")

# Invalid parameters
invalid_params = {
    'theta_E': 5.0,
    'invalid_param': 1.0
}

try:
    SupportedLensModels.validate_parameters('SIE', invalid_params)
    print("✗ Invalid SIE parameters accepted (should be rejected)")
except ValueError as e:
    print(f"✓ Invalid SIE parameters correctly rejected: {e}")


Parameter Validation Test:
✓ Valid SIE parameters accepted
✓ Invalid SIE parameters correctly rejected: Unknown parameter 'invalid_param' for SIE. Valid parameters: ['theta_E', 'e1', 'e2', 'center_x', 'center_y']


## 6. Exporting for Analysis {#export}

Now let's extract the parameters and information needed for analysis.


In [42]:
# Get all parameter names
param_names = lens_input.get_all_parameter_names()
print(f"Total parameters: {len(param_names)}")
print("\nParameter names:")
for i, name in enumerate(param_names):
    print(f"  {i+1:2d}. {name}")


Total parameters: 21

Parameter names:
   1. H0
   2. Om0
   3. lens_0_theta_E
   4. lens_0_e1
   5. lens_0_e2
   6. image_x1
   7. image_y1
   8. image_x2
   9. image_y2
  10. image_x3
  11. image_y3
  12. image_x4
  13. image_y4
  14. source_amp
  15. source_R_sersic
  16. source_n_sersic
  17. source_e1
  18. source_e2
  19. noise_sigma_bkg
  20. zl
  21. zs


In [43]:
# Get fixed parameters
fixed_params = lens_input.get_fixed_parameters()
print(f"Fixed parameters: {len(fixed_params)}")
for name, value in fixed_params.items():
    print(f"  {name}: {value}")


Fixed parameters: 4
  lens_0_center_x: 0.0
  lens_0_center_y: 0.0
  source_center_x: 0.05
  source_center_y: 0.1


In [44]:
# Get lens model information
lens_models = lens_input.get_lens_model_list()
lens_kwargs = lens_input.get_lens_kwargs()
source_kwargs = lens_input.get_source_kwargs()
lens_light_kwargs = lens_input.get_lens_light_kwargs()

print(f"Lens models: {lens_models}")
print(f"\nLens kwargs (2 models):")
for i, kwargs in enumerate(lens_kwargs):
    print(f"  Model {i}: {kwargs}")
print(f"\nSource kwargs: {source_kwargs}")
print(f"\nLens light kwargs: {lens_light_kwargs}")


Lens models: ['SIE']

Lens kwargs (2 models):
  Model 0: {'theta_E': 5.0, 'e1': 0.137, 'e2': 0.039, 'center_x': 0.0, 'center_y': 0.0}

Source kwargs: {'amp': 4.0, 'R_sersic': 0.5, 'n_sersic': 2.0, 'e1': 0.05, 'e2': 0.05, 'center_x': 0.05, 'center_y': 0.1}

Lens light kwargs: {}


In [45]:
# Get parameter values and bounds
param_values = param_input.get_parameter_values()
free_values = param_input.get_free_parameter_values()
fixed_values = param_input.get_fixed_parameter_values()
param_bounds = param_input.get_parameter_bounds()
param_priors = param_input.get_parameter_priors()

print(f"Parameter values: {len(param_values)} total")
print(f"Free parameters: {len(free_values)}")
print(f"Fixed parameters: {len(fixed_values)}")
print(f"Parameters with bounds: {len(param_bounds)}")
print(f"Parameters with priors: {len(param_priors)}")


Parameter values: 41 total
Free parameters: 31
Fixed parameters: 10
Parameters with bounds: 30
Parameters with priors: 30


In [ ]:
# Get parameters by group
cosmology_params = param_input.get_parameters_by_group('cosmology')
lens_params = param_input.get_parameters_by_group('lens')
source_params = param_input.get_parameters_by_group('source')
image_params = param_input.get_parameters_by_group('image_positions')

print(f"Cosmology parameters: {cosmology_params}")
print(f"Lens parameters: {len(lens_params)} parameters")
print(f"Source parameters: {len(source_params)} parameters")
print(f"Image position parameters: {len(image_params)} parameters")


Cosmology parameters: ['H0', 'Om0']
Lens parameters: 12 parameters
Source parameters: 7 parameters
Image position parameters: 8 parameters
